In [20]:
import os
import numpy as np
import shutil
from pathlib import Path

# Function to extract Cα coordinates from a PDB file
def extract_ca_coordinates(pdb_file, chainID):
    ca_coords = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith("ATOM") and " CA " in line and line[21] == chainID:
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                ca_coords.append([x, y, z])
    return np.array(ca_coords)

# Function to calculate RMSD between two sets of coordinates
def calculate_rmsd(coords1, coords2):
    diff = coords1 - coords2
    return np.sqrt(np.sum(diff**2) / len(coords1))

# Function to superpose structures using the Kabsch algorithm
def superpose_structures(ref_coords, target_coords):
    ref_center = np.mean(ref_coords, axis=0)
    target_center = np.mean(target_coords, axis=0)

    ref_centered = ref_coords - ref_center
    target_centered = target_coords - target_center

    H = np.dot(target_centered.T, ref_centered)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)

    aligned_coords = np.dot(target_centered, R) + ref_center
    return aligned_coords

# Function to process a model-seed combination and identify convergence
def process_model_seed(folder, model_number, model_seed, chainID='A', rmsd_threshold=7):
    # Ensure the output folder exists
    converged_folder = Path(folder) / "rmsf_converged"
    converged_folder.mkdir(exist_ok=True)

    # Collect all PDB files for this model and seed
    pdb_files = sorted([
        f for f in os.listdir(folder)
        if f.endswith(".pdb") and f"model_{model_number}_ptm_" in f and f"seed{model_seed}" in f
    ])

    print(f"Processing Model {model_number}, Seed {model_seed}: Found {len(pdb_files)} PDB files.")

    if len(pdb_files) < 2:
        print(f"Not enough recycles for model {model_number}, seed {model_seed}.")
        return

    # Iterate through the recycles and calculate RMSD
    for i in range(len(pdb_files) - 1):
        file1 = Path(folder) / pdb_files[i]
        file2 = Path(folder) / pdb_files[i + 1]

        coords1 = extract_ca_coordinates(file1, chainID)
        coords2 = extract_ca_coordinates(file2, chainID)

        if coords1.size == 0 or coords2.size == 0:
            print(f"Skipping files {pdb_files[i]} or {pdb_files[i + 1]} due to missing Cα coordinates.")
            continue

        aligned_coords2 = superpose_structures(coords1, coords2)
        rmsd = calculate_rmsd(coords1, aligned_coords2)

        # Print the RMSD for this pair of recycles
        print(f"Model {model_number}, Seed {model_seed}, Recycle {i} to {i + 1}: RMSD = {rmsd:.2f} Å")

        # If RMSD is below the threshold, copy the file to the converged folder
        if rmsd <= rmsd_threshold:
            print(f"Convergence detected: RMSD = {rmsd:.2f} Å. Copying files.")
            shutil.copy(file1, converged_folder)
            shutil.copy(file2, converged_folder)
            break

# Function to process all models and seeds in a folder
def process_all_models(folder, rmsd_threshold=7, chainID='A'):
    print(f"Scanning folder: {folder}")
    models_and_seeds = {}
    for filename in os.listdir(folder):
        if filename.endswith(".pdb"):
            parts = filename.split("_")
            try:
                model_number = parts[1]  # Extract the model number
                seed_number = parts[4][4:].split(".")[0]  # Extract the seed number
                if (model_number, seed_number) not in models_and_seeds:
                    models_and_seeds[(model_number, seed_number)] = []
                models_and_seeds[(model_number, seed_number)].append(filename)
            except IndexError:
                print(f"Skipping file {filename}: unexpected filename format.")
                continue

    # Process each model and seed combination
    for (model_number, seed_number), files in models_and_seeds.items():
        process_model_seed(folder, model_number, seed_number, chainID, rmsd_threshold)



In [23]:


# Example usage
if __name__ == "__main__":
    folder_path = '/Users/adrianahernandezgonzalez/LabNotebook/11-24/states/partialAlphaCaV12HS8HLPlocalrun_b3702_8_16_10/pdb/'  # Replace with the path to your folder containing PDB files
    rmsd_cutoff = 12  # Default RMSD cutoff for convergence
    process_all_models(folder_path, rmsd_threshold=rmsd_cutoff)


Scanning folder: /Users/adrianahernandezgonzalez/LabNotebook/11-24/states/partialAlphaCaV12HS8HLPlocalrun_b3702_8_16_10/pdb/
Processing Model 2, Seed 1: Found 13 PDB files.
Model 2, Seed 1, Recycle 0 to 1: RMSD = 71.61 Å
Model 2, Seed 1, Recycle 1 to 2: RMSD = 21.49 Å
Model 2, Seed 1, Recycle 2 to 3: RMSD = 24.92 Å
Model 2, Seed 1, Recycle 3 to 4: RMSD = 59.72 Å
Model 2, Seed 1, Recycle 4 to 5: RMSD = 30.42 Å
Model 2, Seed 1, Recycle 5 to 6: RMSD = 18.46 Å
Model 2, Seed 1, Recycle 6 to 7: RMSD = 8.34 Å
Convergence detected: RMSD = 8.34 Å. Copying files.
Processing Model 3, Seed 7: Found 13 PDB files.
Model 3, Seed 7, Recycle 0 to 1: RMSD = 62.43 Å
Model 3, Seed 7, Recycle 1 to 2: RMSD = 57.96 Å
Model 3, Seed 7, Recycle 2 to 3: RMSD = 62.14 Å
Model 3, Seed 7, Recycle 3 to 4: RMSD = 55.48 Å
Model 3, Seed 7, Recycle 4 to 5: RMSD = 59.30 Å
Model 3, Seed 7, Recycle 5 to 6: RMSD = 54.70 Å
Model 3, Seed 7, Recycle 6 to 7: RMSD = 54.95 Å
Model 3, Seed 7, Recycle 7 to 8: RMSD = 62.73 Å
Model 3,

Model 3, Seed 0, Recycle 8 to 9: RMSD = 60.81 Å
Model 3, Seed 0, Recycle 9 to 10: RMSD = 56.78 Å
Model 3, Seed 0, Recycle 10 to 11: RMSD = 59.19 Å
Model 3, Seed 0, Recycle 11 to 12: RMSD = 61.38 Å
Processing Model 2, Seed 6: Found 13 PDB files.
Model 2, Seed 6, Recycle 0 to 1: RMSD = 53.94 Å
Model 2, Seed 6, Recycle 1 to 2: RMSD = 20.70 Å
Model 2, Seed 6, Recycle 2 to 3: RMSD = 14.49 Å
Model 2, Seed 6, Recycle 3 to 4: RMSD = 58.49 Å
Model 2, Seed 6, Recycle 4 to 5: RMSD = 23.94 Å
Model 2, Seed 6, Recycle 5 to 6: RMSD = 14.94 Å
Model 2, Seed 6, Recycle 6 to 7: RMSD = 14.54 Å
Model 2, Seed 6, Recycle 7 to 8: RMSD = 21.02 Å
Model 2, Seed 6, Recycle 8 to 9: RMSD = 12.18 Å
Model 2, Seed 6, Recycle 9 to 10: RMSD = 11.57 Å
Model 2, Seed 6, Recycle 10 to 11: RMSD = 11.75 Å
Model 2, Seed 6, Recycle 11 to 12: RMSD = 12.80 Å
Processing Model 3, Seed 8: Found 13 PDB files.
Model 3, Seed 8, Recycle 0 to 1: RMSD = 50.47 Å
Model 3, Seed 8, Recycle 1 to 2: RMSD = 49.94 Å
Model 3, Seed 8, Recycle 2 to 

Model 1, Seed 2, Recycle 10 to 11: RMSD = 6.72 Å
Convergence detected: RMSD = 6.72 Å. Copying files.
Processing Model 5, Seed 3: Found 13 PDB files.
Model 5, Seed 3, Recycle 0 to 1: RMSD = 60.37 Å
Model 5, Seed 3, Recycle 1 to 2: RMSD = 25.62 Å
Model 5, Seed 3, Recycle 2 to 3: RMSD = 25.24 Å
Model 5, Seed 3, Recycle 3 to 4: RMSD = 80.02 Å
Model 5, Seed 3, Recycle 4 to 5: RMSD = 34.17 Å
Model 5, Seed 3, Recycle 5 to 6: RMSD = 16.82 Å
Model 5, Seed 3, Recycle 6 to 7: RMSD = 26.60 Å
Model 5, Seed 3, Recycle 7 to 8: RMSD = 15.25 Å
Model 5, Seed 3, Recycle 8 to 9: RMSD = 28.33 Å
Model 5, Seed 3, Recycle 9 to 10: RMSD = 7.87 Å
Convergence detected: RMSD = 7.87 Å. Copying files.
Processing Model 4, Seed 5: Found 13 PDB files.
Model 4, Seed 5, Recycle 0 to 1: RMSD = 68.02 Å
Model 4, Seed 5, Recycle 1 to 2: RMSD = 61.03 Å
Model 4, Seed 5, Recycle 2 to 3: RMSD = 51.97 Å
Model 4, Seed 5, Recycle 3 to 4: RMSD = 53.22 Å
Model 4, Seed 5, Recycle 4 to 5: RMSD = 63.90 Å
Model 4, Seed 5, Recycle 5 to 6